In [3]:
cd /Users/martin/Git/estates

/Users/martin/Git/estates


In [2]:
import os
import boto3
import json
from decimal import Decimal
import pandas as pd

In [4]:
file = 'data/bronze/20210703-104535_1038932572.json'

In [5]:
with open(file) as f:
    data = json.load(f)

In [6]:
columns = [
    # Web
    'estate_id',
    'timestamp',
    'estate_title', 
    'estate_description_short',
    'estate_description_long',
    'estate_longitude',
    'estate_latitude',
    'estate_map_zoom',
    'estate_images',
    'seller_ico',
    'seller_email',
    'seller_numbers',
    'seller_web',
    'seller_address',
    'seller_name',
    
    # Target
    'Celková cena',

    # Features
    'Užitná plocha',
    'Podlaží',
    'estate_locality_district',
    'estate_disposition',
    'estate_category_main_cb',
    'Stavba',
    'Stav objektu',
    'Poznámka k ceně',
    'Energetická náročnost budovy',
    'Vlastnictví',
    'Tram',
    'Vybavení',
    'Výtah',
    'Divadlo',
    'Kino',
    'Obchod',
    'Cukrárna',
    'Veterinář',
    'Vlak',
    'Lékárna',
    'Bankomat',
    'Sportoviště',
    'Bus MHD',
    'Lékař',
    'Škola',
    'Školka',
    'Hospoda',
    'Pošta',
    'Restaurace',
    'Večerka',
    'Hřiště',
]

In [7]:
data = {
    _id: {item: value for item, value in items.items() if item in columns}
    for _id, items in data.items()
}
ddb_data = json.loads(json.dumps(data), parse_float=Decimal)

In [12]:
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('estates')

with table.batch_writer() as batch:
    for _id, items in ddb_data.items():
        batch.put_item(
            Item={
                'estate_id': _id,
                'items': items
            }
        )